# 00. 모듈 설치

In [1]:
# !pip install kiwipiepy

In [2]:
import numpy as np
import pandas as pd

import re
from collections import Counter, defaultdict
from functools import partial
from tqdm import tqdm

In [3]:
pwd

'c:\\Users\\user\\Desktop\\PinkWink_NLK_NLP_hackerthon\\FINAL'

In [4]:
ls

 C ����̺��� ����: BOOTCAMP
 ���� �Ϸ� ��ȣ: 0246-9FF7

 c:\Users\user\Desktop\PinkWink_NLK_NLP_hackerthon\FINAL ���͸�

2023-01-26  ���� 07:40    <DIR>          .
2023-01-26  ���� 07:40    <DIR>          ..
2023-01-26  ���� 07:50            23,486 01_.ipynb
2023-01-26  ���� 03:37               515 02_.ipynb
2023-01-26  ���� 07:40    <DIR>          data
2023-01-26  ���� 07:40    <DIR>          module
               2�� ����              24,001 ����Ʈ
               4�� ���͸�  92,454,354,944 ����Ʈ ����


In [5]:
from module.preprocessing import PreProcessing, TokenKiwi
from module.collection import Crawling

# 01. 데이터 불러오기

In [6]:
test_data = pd.read_csv('../data/sample_data_오더퀸.csv')
train_data = pd.read_csv('../data/sample_data_우노스.csv')

In [7]:
food = pd.read_excel('../data/food.xlsx')

In [8]:
test = test_data[['점포명', 'MENU_FULL_NM', '판매가격']]
train = train_data[['매장명', '메뉴명', '가격']]
train['표준명'] = train['메뉴명']

In [9]:
from module.tg import ss, Dictionary

In [10]:
s = ss()

In [11]:
train = s.process(train, train=True)
test = s.process(test, train=False)

store_to_sector


100%|██████████| 6/6 [00:00<?, ?it/s]


crawling_to_sector
store_to_sector


100%|██████████| 4/4 [00:00<00:00,  6.99it/s]

crawling_to_sector


In [12]:
dictionary = Dictionary

In [13]:
train

,STORE,MENU,PRICE,STANDARD,SECTOR
0,포케올데이_시흥은계점,메인토핑추가,2000,메인토핑추가,도시락
1,포케올데이_시흥은계점,아보카도추가,2000,아보카도추가,도시락
2,포케올데이_시흥은계점,또띠아빵추가,1500,또띠아빵추가,도시락
3,포케올데이_시흥은계점,현미밥추가,1000,현미밥추가,도시락
4,포케올데이_시흥은계점,메밀면추가,1000,메밀면추가,도시락
...,...,...,...,...,...
463,오가샐러드_중화산점,1800-4520에2번,2000,1800-4520에2번,카페
464,오가샐러드_중화산점,테스트결제는가능합니다,100,테스트결제는가능합니다,카페
465,오가샐러드_중화산점,(ICE)레몬에이드,5000,(ICE)레몬에이드,카페
466,오가샐러드_중화산점,된장스무디,80000,된장스무디,카페


In [14]:
from module.jh import JH

In [15]:
jh = JH()

In [26]:
type(test['MENU'].apply(jh.morph))

pandas.core.series.Series

In [43]:
test['OPTION'] = test['MENU'].apply(jh.morph).str.get(1)
test['MENU'] = test['MENU'].apply(jh.morph).str.get(0)

In [17]:
kiwi = TokenKiwi('s')

In [18]:
wordls = []
for menu in train['STANDARD'].astype(str):
    for word in menu.split(' '):
        pattern = r'[^\w]'         # 특수기호제거
        word = re.sub(pattern, '', word)
        wordls.append(word)
wordls = list(set(wordls))

In [19]:
# kiwi.dictionary_add(wordls)
kiwi.dictionary_add(wordls, type='w')

In [20]:
kiwi.kiwi.tokenize('믹스 조금매운맛')

[Token(form='믹스', tag='NNG', start=0, len=2),
 Token(form='조금', tag='MAG', start=3, len=2),
 Token(form='맵', tag='VA-I', start=5, len=2),
 Token(form='은', tag='ETM', start=6, len=1),
 Token(form='맛', tag='NNG', start=7, len=1)]

- 비지도 학습
- 단어 간 유사도가 높다고 판단 조건
    - 문자열 길이가 같은 경우
        1. 단순 단어 유사도
        2. 자모음 분리 후 유사도
        3. 초성끼리 중성끼리 같던가
    - 문자열 길이가 다른 경우
        1. 단순 단어 유사도
        2. 자모음 분리 후 유사도

In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer

In [21]:
# !pip install jamo

In [70]:
from jamo import h2j, j2hcj

In [71]:
def h2cj(text):
    return j2hcj(h2j(text))

In [75]:
X_tfidf.shape

(360, 356)

In [82]:
tfidf = TfidfVectorizer()
# X_tfidf = tfidf.fit_transform(wordls)
X_tfidf = tfidf.fit_transform(list(map(h2cj, wordls)))
nor = Normalizer()
X_tfidf = nor.fit_transform(X_tfidf)

In [90]:
from sklearn.cluster import DBSCAN

dbs = DBSCAN(eps=0.5, min_samples=2)
dbs.fit(X_tfidf)
label_dbs = dbs.labels_

print(label_dbs)

[ 0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1

In [91]:
for menu, label in zip(wordls, label_dbs):
    if label != -1:
        print(menu)


2
1
ice
Ice


In [101]:
pp = PreProcessing()

In [103]:
## 기존 trian 데이터에서 메뉴명, 표준명 가져오기
for menu, standard in zip(train['MENU'], train['STANDARD']):
    # for menu, standard in zip(train['MENU'][:10], train['STANDARD'][:10]):
    # print(menu, standard)
    key = standard
    value = [menu]
    
    # 실제로 할떄는 제외
    step_input = [pp.text_clean(menu), pp.text_clean(standard)]
    step_output = []
    for words in step_input:
        step_output.append(words)
        step_output.append(kiwi.lambda_kiwi_spacing(words))
        # step_output.append(' '.join(okt.morphs(words)))
    
    step_input = list(set(step_output))
    step_output = []
    for words in step_input:
        ls = []
        for word in words.split(' '):
            try:
                ls.append(dictionary.word_dict_map[word])
            except:
                ls.append(word)
        step_output.append(' '.join(ls))
    
    # print(step_output)
    # step_input = list(set(step_output))
    # step_output = []
    # for words in step_input:
    #     ls = words.split(' ')
    #     ls = list(map(list, permutations(ls, len(ls))))
        
    #     for l in ls:
    #         step_output.append(' '.join(l))
    
    # print(step_output)   
    step_input = list(set(step_output))
    step_output = []
    for words in step_input:
        step_output.append(words)
        step_output.append(''.join(words.split(' ')))
    # print(step_output)
    
    dictionary.standard_dict[key] = list(set(step_output))
    for words in list(set(step_output)):
        dictionary.standard_dict_map[words] = key

In [104]:
dictionary.standard_dict_map

{'메인토핑추가': '메인토핑추가',
 '아보카도 추가': '아보카도추가',
 '아보카도추가': '아보카도추가',
 '또띠아빵추가': '또띠아빵추가',
 '현미밥추가': '현미밥추가',
 '현미밥 추가': '현미밥추가',
 '메밀면추가': '메밀면추가',
 '메밀면 추가': '메밀면추가',
 '캔들판매': '캔들판매',
 '멤버쉽가입': '멤버쉽 가입',
 '멤버쉽 가입': '멤버쉽 가입',
 '정기구독 101': '정기구독 10+1',
 '정기구독101': '정기구독 10+1',
 '정기구독 152': '정기구독 15+2',
 '정기구독152': '정기구독 15+2',
 '텀블러': '텀블러',
 '현미밥 포케': '현미밥 포케',
 '현미밥포케': '현미밥 포케',
 '메밀면포케': '메밀면 포케',
 '메밀면 포케': '메밀면 포케',
 '채소만포케': '채소만 포케',
 '채소만 포케': '채소만 포케',
 '커피 세트': '커피 세트',
 '커피세트': '커피 세트',
 '티 세트': '티 세트',
 '티세트': '티 세트',
 '곤약냉우동': '곤약 냉우동',
 '곤약 냉우동': '곤약 냉우동',
 '나홀로하와이세트': '나홀로 하와이 세트',
 '나홀로 하와이 세트': '나홀로 하와이 세트',
 '둘이서 하와이 세트': '둘이서 하와이 세트',
 '둘이서하와이세트': '둘이서 하와이 세트',
 '나홀로포케미니곤약냉우동세트': '나홀로 포케+미니곤약냉우동 세트',
 '나홀로 포케미니곤약냉우동 세트': '나홀로 포케+미니곤약냉우동 세트',
 '미트랩': '미트랩',
 '씨랩': '씨랩',
 '두부볼2개': '두부볼 2개',
 '두부볼 2개': '두부볼 2개',
 '단호박크림수프': '단호박 크림수프',
 '단호박 크림수프': '단호박 크림수프',
 '배달비100': '배달비 100',
 '배달비 100': '배달비 100',
 '배달비 500': '배달비 500',
 '배달비500': '배달비 500',
 '배달비1000': '배달비 1000',
 '